# Mesa Schelling example - Schelling Segregation Model

[[Code explanation]](https://towardsdatascience.com/introduction-to-mesa-agent-based-modeling-in-python-bcb0596e1c9a) **Note that the final interactive visualization part we will cover in the later parts of the course**

## Background

The Schelling (1971) segregation model is a classic of agent-based modeling, demonstrating how agents following simple rules lead to the emergence of qualitatively different macro-level outcomes. Agents are randomly placed on a grid. There are two types of agents, one constituting the majority and the other the minority. All agents want a certain number (generally, 3) of their 8 surrounding neighbors to be of the same type in order for them to be happy. Unhappy agents will move to a random available grid space. While individual agents do not have a preference for a segregated outcome (e.g. they would be happy with 3 similar neighbors and 5 different ones), the aggregate outcome is nevertheless heavily segregated.

# 1. Create the Basic Agent/Model

In [ ]:
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid

In [ ]:
# Agent

class SchellingAgentBasic(Agent):
    
    # 1 Initialization
    def __init__(self, pos, model, agent_type):
        
        '''
         Create a new Schelling agent.

         Args:
            unique_id: Unique identifier for the agent.
            x, y: Agent initial location.
            agent_type: Indicator for the agent's type (minority=1, majority=0)
        '''
        
        super().__init__(pos, model)
        self.pos = pos
        self.type = agent_type
        
    # 2 Step function
    def step(self):
        similar = 0
        neighbors = self.model.grid.neighbor_iter(self.pos)
        
        # 3 Calculate the number of similar neighbours
        for neighbor in neighbors:
            if neighbor.type == self.type:
                similar += 1

        # 4 Move to a random empty location if unhappy
        if similar < self.model.homophily:
            self.model.grid.move_to_empty(self)
        else:
            self.model.happy += 1

In [ ]:
# Model

class SchellingModelBasic(Model):
    '''
    Model class for the Schelling segregation model.
    '''

    def __init__(self, height, width, density, minority_pc, homophily):
        '''
        Create a new Schelling model.

         Args:
            width: Horizontal axis of the grid which is used together with Height to define the total number of agents in the system.
            height: Vertical axis of the grid which is used together with Width to define the total number of agents in the system.
            density: Define the population density of agent in the system. Floating value from 0 to 1.
            fraction minority: The ratio between blue and red. Blue is represented as the minority while red is represented as the majority. Floating value from 0 to 1. If the value is higher than 0.5, blue will become the majority instead.
            homophily: Define the number of similar neighbors required for the agents to be happy. Integer value range from 0 to 8 since you can only be surrounded by 8 neighbors.
        '''
        super().__init__()
        self.height = height
        self.width = width
        self.density = density
        self.minority_pc = minority_pc
        self.homophily = homophily

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(height, width, torus=True)

        self.happy = 0

        self.running = True

        # Set up agents
        # We use a grid iterator that returns
        # the coordinates of a cell as well as
        # its contents. (coord_iter)
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            if self.random.random() < self.density:
                if self.random.random() < self.minority_pc:
                    agent_type = 1
                else:
                    agent_type = 0

                agent = SchellingAgentBasic((x, y), self, agent_type)
                self.grid.position_agent(agent, (x, y))
                self.schedule.add(agent)

    def step(self):
        '''
        Run one step of the model. If All agents are happy, halt the model.
        '''
        self.happy = 0  # 1 Reset counter of happy agents
        self.schedule.step()
        
        # 2 Stop the model if all agents are happy
        if self.happy == self.schedule.get_agent_count():
            self.running = False

# 2. Run the Agent/Model Basic

<font color='green'>**HINT:** Now we instantiate a model instance: a 10x10 grid, with an 80% chance of an agent being placed in each cell, approximately 20% of agents set as minorities, and agents wanting at least 3 similar neighbors.</font>

In [ ]:
model = SchellingModelBasic(10, 10, 0.8, 0.2, 3)
for i in range(100):
    model.step()

print(model.schedule.steps) # Show how many steps have actually run

# 3. Visualize the Agent/Model

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer


def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 0.5}
    
    if agent.type == 0:
        portrayal["Color"] = "Red"
    else:
        portrayal["Color"] = "Blue"
        
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(SchellingModelBasic,
                       [grid],
                       "Schelling Model",
                       {"density":0.8, "width":10, "height":10, "minority_pc":0.2, "homophily":3})

# 4. Run the Agent/Model Visualization

In [ ]:
server.port = 8521 # The default
#server.launch()

# 5. Collect data to Analyze the Agent/Model

In [ ]:
# Data collection

def get_model_analysis_data(model):
    '''
    Find the % of agents that only have neighbors of their same type.
    '''
    segregated_agents = 0
    for agent in model.schedule.agents:
        segregated = True
        for neighbor in model.grid.neighbor_iter(agent.pos):
            if neighbor.type != agent.type:
                segregated = False
                break
        if segregated:
            segregated_agents += 1
    return segregated_agents / model.schedule.get_agent_count()

In [ ]:
# Agent

class SchellingAgentAnalysis(Agent):
    '''
    Schelling segregation agent
    '''
    
    # 1 Initialization
    def __init__(self, pos, model, agent_type):
        '''
         Create a new Schelling agent.

         Args:
            unique_id: Unique identifier for the agent.
            x, y: Agent initial location.
            agent_type: Indicator for the agent's type (minority=1, majority=0)
        '''
        super().__init__(pos, model)
        self.pos = pos
        self.type = agent_type

    # 2 Step function
    def step(self):
        similar = 0
        neighbors = self.model.grid.neighbor_iter(self.pos)
        
        # 3 Calculate the number of similar neighbours
        for neighbor in neighbors:
            if neighbor.type == self.type:
                similar += 1

        # 4 Move to a random empty location if unhappy
        if similar < self.model.homophily:
            self.model.grid.move_to_empty(self)
        else:
            self.model.happy += 1

In [ ]:
from mesa.datacollection import DataCollector

# Model

class SchellingModelAnalysis(Model):
    '''
    Model class for the Schelling segregation model.
    '''

    def __init__(self, height, width, density, minority_pc, homophily):
        '''
        Create a new Schelling model.

         Args:
            width: Horizontal axis of the grid which is used together with Height to define the total number of agents in the system.
            height: Vertical axis of the grid which is used together with Width to define the total number of agents in the system.
            density: Define the population density of agent in the system. Floating value from 0 to 1.
            fraction minority: The ratio between blue and red. Blue is represented as the minority while red is represented as the majority. Floating value from 0 to 1. If the value is higher than 0.5, blue will become the majority instead.
            homophily: Define the number of similar neighbors required for the agents to be happy. Integer value range from 0 to 8 since you can only be surrounded by 8 neighbors.
        '''
        super().__init__()
        self.height = height
        self.width = width
        self.density = density
        self.minority_pc = minority_pc
        self.homophily = homophily

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(height, width, torus=True)

        self.happy = 0
        self.datacollector = DataCollector(
            {"Happy": "happy", "Segregated_Agents": get_model_analysis_data},  # Model-level count of happy agents
            # For testing purposes, agent's individual x and y
            {"x": lambda a: a.pos[0], "y": lambda a: a.pos[1]})

        self.running = True

        # Set up agents
        # We use a grid iterator that returns
        # the coordinates of a cell as well as
        # its contents. (coord_iter)
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            if self.random.random() < self.density:
                if self.random.random() < self.minority_pc:
                    agent_type = 1
                else:
                    agent_type = 0

                agent = SchellingAgentAnalysis((x, y), self, agent_type)
                self.grid.position_agent(agent, (x, y))
                self.schedule.add(agent)

    def step(self):
        '''
        Run one step of the model. If All agents are happy, halt the model.
        '''
        self.happy = 0  # 1 Reset counter of happy agents
        self.schedule.step()
        
        # 2 collect data
        self.datacollector.collect(self)
        
        # 3 Stop the model if all agents are happy
        if self.happy == self.schedule.get_agent_count():
            self.running = False

# 6. Run the Agent/Model Analysis

In [ ]:
model = SchellingModelAnalysis(10, 10, 0.8, 0.2, 3)
for i in range(100):
    model.step()

print(model.schedule.steps) # Show how many steps have actually run

In [ ]:
import pandas as pd

model_out = model.datacollector.get_model_vars_dataframe()
model_out.head()

In [ ]:
model_out.Happy.plot()

In [ ]:
agent_out = model.datacollector.get_agent_vars_dataframe()
agent_out.head()

In [ ]:
agent_out.xs((1,4), level="AgentID").plot()

# 7. Create iteration Batch of the Agent/Model

In [ ]:
params = {"height": 10, "width": 10, "density": 0.8, "minority_pc": 0.2, 
              "homophily": range(1,9)}

# 8. Run the Agent/Model Batch

In [ ]:
from mesa.batchrunner import batch_run

results = batch_run(
    SchellingModelAnalysis,
    parameters=params,
    iterations=10,
    max_steps=200,
    display_progress=True,
)

# 9. Run the Batch data Analysis

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.head()

Task: Find out how homophily (level of neighbour similaritly) influences the final segragation of agents using the mean or box plot. You should be able to plot the average outcome for each homophily value.

**hint** Your plot should look similar to [this paper](https://www.jasss.org/15/1/6.html). Not neccesarily with Moran's I, but the transition should be visible

In [ ]:
results_df.groupby(by=["RunId"]).median().boxplot(by ='homophily', column =['Segregated_Agents'], grid=False)

# Appendix

# 3. Visualize the Agent/Model - interactive advanced

In [ ]:
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter

class HappyElement(TextElement):
    '''
    Display a text count of how many happy agents there are.
    '''

    def render(self, model):
        return "Happy agents: " + str(model.happy)


def agent_portrayal(agent):
    '''
    Portrayal Method for canvas
    '''
    if agent is None:
        return
    portrayal = {"Shape": "circle", "r": 0.5, "Filled": "true", "Layer": 0}

    if agent.type == 0:
        portrayal["Color"] = "Red"
    else:
        portrayal["Color"] = "Blue"
        
    return portrayal

happy_element = HappyElement()
canvas_element = CanvasGrid(agent_portrayal, 20, 20, 500, 500)
happy_chart = ChartModule([{"Label": "Happy", "Color": "Black"}], data_collector_name="datacollector")

model_params = {
    "density":0.8, 
    "width":20, 
    "height":20, 
    "minority_pc":0.2, 
    "homophily":UserSettableParameter(
        "slider",
        "Number of similar agents",
        3,
        1,
        8,
        1,
        description="Choose how many similar agents needed to be happy",
    ),
}

server = ModularServer(SchellingModelAnalysis,
                       [canvas_element, happy_element, happy_chart],
                       "Schelling’s Segregation Model",
                       model_params)

In [ ]:
server.port = 8521 # The default
#server.launch()